In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
703,2024-07-02,Venezuela Superliga,21:00,Panteras,Diablos,1.19,4.37,151.5,1.84,1.86,0.0,0.00,0.00,dICdwD2D,0.840336,0.228833,0.543478,0.537634,0.069169,214.946,74.472688,0.346472,0.6,1.341641,2.236068,216.72,3.062,0.872336,0.284891,-26.0,322.924,104.683586,0.324174,0.6,1.341641,2.236068,472.78,4.064,1.466383,0.360823,-21.0,63,77,3.44,6.14,176.006,312.162,0.808849,0.007644,NaN,-2.47,-0.494,-0.384615,0.000000,0.0,0.000000,1.14,0.228,14.780702,0.000000,0.0,0.000000
704,2024-07-02,Venezuela Superliga,18:00,Spartans,Toros de Aragua,1.14,5.19,167.5,1.80,1.87,0.0,0.00,0.00,z3OmuZX0,0.877193,0.192678,0.555556,0.534759,0.069871,163.310,65.908095,0.403577,1.8,1.643168,0.912871,106.50,2.004,0.665079,0.331876,13.0,366.546,146.223581,0.398923,0.0,0.000000,NaN,415.20,4.764,2.038095,0.427812,-67.0,75,80,1.42,5.19,180.838,359.898,0.904829,0.026974,NaN,-1.60,-0.320,-0.437500,0.000000,0.0,0.000000,-5.00,-1.000,-4.190000,0.000000,0.0,0.000000
705,2024-07-02,Singapura Nbl,09:30,Tagawa,SBA,1.41,2.76,131.5,1.85,1.85,0.0,0.00,0.00,GpWgPwvH,0.709220,0.362319,0.540541,0.540541,0.071539,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.457839,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
706,2024-07-02,Singapura Nbl,08:00,Xin Hua,Adroit,9.40,1.04,138.5,1.83,1.83,0.0,0.00,0.00,CQ4689Fb,0.106383,0.961538,0.546448,0.546448,0.067921,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.132455,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
707,2024-07-02,República Dominicana Lnb,21:00,Reales de la Vega,Indios,1.55,2.34,173.5,1.85,1.85,0.0,0.00,0.00,xnPwYX5S,0.645161,0.427350,0.540541,0.540541,0.072512,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,77,1.27,1.31,0.000,0.000,0.287205,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
708,2024-07-02,República Dominicana Lnb,21:00,Caneros,Leones,1.61,2.22,160.5,1.87,1.83,0.0,0.00,0.00,4YQVZgyG,0.621118,0.450450,0.534759,0.546448,0.071568,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,62,0,2.68,0.00,0.000,0.000,0.225240,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
709,2024-07-02,Eua Wnba,23:00,Los Angeles Sparks F,Washington Mystics F,2.04,1.78,161.5,1.83,1.95,1.5,1.89,1.63,Gv11f4PJ,0.490196,0.561798,0.546448,0.512821,0.051994,458.704,167.899393,0.366030,0.0,0.000000,NaN,608.00,6.036,2.148600,0.355964,-56.0,302.768,169.856975,0.561014,0.6,1.341641,2.236068,504.35,3.656,2.086068,0.570588,-21.0,80,77,7.60,6.55,168.950,352.150,0.096255,0.044896,0.104459,-1.22,-0.244,-4.262295,0.000000,0.0,0.000000,-5.00,-1.000,-0.780000,0.000000,0.0,0.000000
710,2024-07-02,Eua Wnba,22:30,Las Vegas Aces F,Indiana Fever F,1.10,6.97,173.5,1.83,1.95,0.0,0.00,0.00,EZMIVKnJ,0.909091,0.143472,

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
